# Dependences

In [ ]:
include("../../src/struct_data.jl")

# Model

In [ ]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

# Aggregate

In [ ]:
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.01)
agg_size = 15

In [ ]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)
# show_aggregates(agg)

# kNN

In [ ]:
include("../../src/neighbor.jl")
println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

# Force

In [ ]:
include("../../src/forces/forces.jl")
# test = zeros(size(agg.Position)) |> cu

for i = 1:50
    threads=(64,3)
    @cuda(
        threads = threads,
        blocks = cld.(size(agg.Position,),threads),
        sum_force!(
            agg.Position,
            agg.Simulation.Force.F,
            agg.Simulation.Force.Pol,
            agg.Simulation.Force.N_i,
            agg.Simulation.Neighbor.idx_sum,
            agg.Simulation.Neighbor.idx_red,
            agg.Simulation.Parameter.Force,
            agg.Simulation.Parameter.Contractile.fₚ,
            atan(1),
            pi/4,
            model.Time.dt
        )
    )
end

display(agg.Position')
display(agg.Simulation.Force.F')
display(agg.Simulation.Force.Pol')
display(agg.Simulation.Force.N_i)

# using Plots

# M = 0
# FOMO = Matrix(test[:,1]')
# for i=unique(FOMO)
#     if i == 0
#         global M = [i sum(FOMO .== i)]
#     else
#         global M = vcat(M, [i sum(FOMO .== i)])
#     end 
# end

# M = sortslices(M,dims=1,by=x->(x[1],-x[2]),rev=false)
# plot(M[:,1],M[:,2])

In [ ]:
# open(model.Output.name_output*".xyz", "w") do f
#     write(f, "$(size(agg.Position, 1))\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Position)), ' ')
# end

# title = "TEST"

In [ ]:
# @showprogress "$(title)..." for t=0:Int(model.Time.tₛᵢₘ/model.Time.dt)
#     # println(t)
#     # CUDA.@time 
#     threads=(64,3)
#     @cuda(
#         threads = threads,
#         blocks = cld.(size(agg.Position,),threads),
#         sum_force!(
#             agg.Position,
#             agg.Simulation.Force.F,
#             agg.Simulation.Force.Pol,
#             agg.Simulation.Force.N_i,
#             agg.Simulation.Neighbor.idx_sum,
#             agg.Simulation.Neighbor.idx_red,
#             agg.Simulation.Parameter.Force,
#             agg.Simulation.Parameter.Contractile.fₚ,
#             atan(1),
#             pi/4,
#             model.Time.dt
#         )
#     )

#     if t%(model.Time.nₖₙₙ) == 0
#         # println("▲ Neighbor")
#         threads=(100)
#         @cuda(
#             threads=threads,
#             blocks=cld.(size(agg.Position,1),threads),
#             dist_kernel!(
#                 agg.Simulation.Neighbor.idx_red,
#                 agg.Simulation.Neighbor.idx_cont,
#                 agg.Simulation.Neighbor.idx_sum,
#                 agg.Simulation.Neighbor.dist,
#                 agg.Position,
#                 agg.Simulation.Parameter.Force.rₘₐₓ
#             )
#         ) 
#     end

#     if t%Int(model.Time.tₛᵢₘ/model.Time.nₛₐᵥₑ) == 0
#         # println("▲ Save")
#         open(model.Output.name_output*".xyz", "a") do f
#             write(f, "$(size(agg.Position, 1))\n")
#             write(f, "t=$(t)\n")
#             writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Position)), ' ')
#         end
#     end
# end

## Function

In [ ]:
# include("../../src/forces/forces.jl")
# test = zeros(1,size(agg.Position,1)) |> cu

# threads=(64,3)
# @cuda(
#     threads = threads,
#     blocks = cld.(size(agg.Position,),threads),
#     sum_force!(
#         agg.Position,
#         agg.Simulation.Force.F,
#         agg.Simulation.Force.Pol,
#         agg.Simulation.Force.N_i,
#         agg.Simulation.Neighbor.idx_sum,
#         agg.Simulation.Neighbor.idx_red,
#         agg.Simulation.Parameter.Force,
#         agg.Simulation.Parameter.Contractile.fₚ,
#         atan(1),
#         pi/4,
#         model.Time.dt,
#         test
#     )
# )

# # display(sum(agg.Simulation.Force.Pol.^2, dims=2)')
# # display(sum(.!(0.0 .< agg.Simulation.Force.Pol .< 1.0),dims=1)')
# # display("--------------------------------------------------------------------------------")
# display(agg.Position')
# display(agg.Simulation.Force.F')
# display(agg.Simulation.Force.Pol')
# display(agg.Simulation.Force.N_i)
# display(test)

# # open("Pol.xyz", "w") do f
# #     write(f, "$(size(agg.Simulation.Force.Pol, 1))\n")
# #     write(f, "t=0\n")
# #     writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Simulation.Force.Pol)), ' ')
# # end

# # open("Force.xyz", "w") do f
# #     write(f, "$(size(agg.Simulation.Force.F, 1))\n")
# #     write(f, "t=0\n")
# #     writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Simulation.Force.F)), ' ')
# # end


# # using Plots

# # M = 0
# # FOMO = Matrix(agg.Simulation.Force.F)[:,1]
# # for i=unique(FOMO)
# #     if i == 0
# #         global M = [i sum(FOMO .== i)]
# #     else
# #         global M = vcat(M, [i sum(FOMO .== i)])
# #     end 
# # end

# # plot(M[:,1],M[:,2])